In [1]:
from google.colab import drive
drive.mount('/content/drive')

KeyboardInterrupt: ignored

In [4]:
pip install pomegranate

In [ ]:
!pip install dipy

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# import plotly_express as px
# px.set_mapbox_access_token("pk.eyJ1Ijoic2hha2Fzb20iLCJhIjoiY2plMWg1NGFpMXZ5NjJxbjhlM2ttN3AwbiJ9.RtGYHmreKiyBfHuElgYq_w")

In [6]:
road_data_path = '/content/drive/My Drive/2021 Route Prediction/Project-1/Source-Code/data/route_network/clustered'
trips_cluster_data_path = '/content/drive/My Drive/2021 Route Prediction/Project-1/Source-Code/data/3.1_cluster_on_trips'
train_path = '/content/drive/My Drive/2021 Route Prediction/Project-1/Source-Code/model/3.1'

In [ ]:
# data_path = '/content/drive/My Drive/Colab Notebooks/RoutePredict/Data'
# module_path = '/content/drive/My Drive/Colab Notebooks/RoutePredict/Module'

# cross validation

In [7]:
days = range(1,32)
month = 1

for day in days:

  filename = trips_cluster_data_path + f'/trips_2019_{month}_{day}.csv'
  print(filename)

/content/drive/My Drive/2021 Route Prediction/Project-1/Source-Code/data/3.1_cluster_on_trips/trips_2019_1_1.csv
/content/drive/My Drive/2021 Route Prediction/Project-1/Source-Code/data/3.1_cluster_on_trips/trips_2019_1_2.csv
/content/drive/My Drive/2021 Route Prediction/Project-1/Source-Code/data/3.1_cluster_on_trips/trips_2019_1_3.csv
/content/drive/My Drive/2021 Route Prediction/Project-1/Source-Code/data/3.1_cluster_on_trips/trips_2019_1_4.csv
/content/drive/My Drive/2021 Route Prediction/Project-1/Source-Code/data/3.1_cluster_on_trips/trips_2019_1_5.csv
/content/drive/My Drive/2021 Route Prediction/Project-1/Source-Code/data/3.1_cluster_on_trips/trips_2019_1_6.csv
/content/drive/My Drive/2021 Route Prediction/Project-1/Source-Code/data/3.1_cluster_on_trips/trips_2019_1_7.csv
/content/drive/My Drive/2021 Route Prediction/Project-1/Source-Code/data/3.1_cluster_on_trips/trips_2019_1_8.csv
/content/drive/My Drive/2021 Route Prediction/Project-1/Source-Code/data/3.1_cluster_on_trips/tr

In [8]:
dfs = []
day_len = []
day_trip_count = []
days = range(1,32)
# days = range(1,2)
months = range(1,4)
# months = range(1,2)

for month in months:
  for day in days:
    try:
      sp_seq_df = pd.read_csv(trips_cluster_data_path + f'/lebeled/trips_2019_{month}_{day}.csv', parse_dates=['time_stamp'])
    except:
      pass

    sp_seq_df = sp_seq_df[sp_seq_df['cluster']>-1].sort_values(by=['trip_id', 'time_stamp'])

    trip_count = sp_seq_df.trip_id.value_counts()
    trip_count = trip_count[trip_count >= 12]
    sp_seq_df = sp_seq_df[sp_seq_df.trip_id.isin(trip_count.index)]
    sp_seq_df['month'] = month
    sp_seq_df['day'] = day
    day_len.append(len(sp_seq_df))
    day_trip_count.append(len(trip_count))
    dfs.append(sp_seq_df)
print(day_len)
print(day_trip_count)
df = pd.concat(dfs, ignore_index=True)
df

df = df[['trip_id', 'sp', 'day', 'month','route_cluster']]
df.head()

,time_stamp,unit_id,lat,lon,speed,unit_type,trip_id,cluster
0,2019-01-01 00:00:07,02800040000000GBBN601005123,16.481607,104.049705,71,3.0,0,-1
1,2019-01-01 00:01:07,02800040000000GBBN601005123,16.491331,104.052101,81,3.0,0,-1
2,2019-01-01 00:02:07,02800040000000GBBN601005123,16.502571,104.054756,65,3.0,0,-1
3,2019-01-01 00:03:07,02800040000000GBBN601005123,16.514196,104.056266,79,3.0,0,-1
4,2019-01-01 00:04:07,02800040000000GBBN601005123,16.524534,104.056427,65,3.0,0,-1
...,...,...,...,...,...,...,...,...
195857,2019-01-01 21:51:58,003000300000359769033400604,14.835272,101.550918,52,7.0,2098,-1
195858,2019-01-01 21:53:56,003000300000359769033400604,14.846673,101.562518,61,7.0,2098,-1
195859,2019-01-01 21:56:00,003000300000359769033400604,14.850647,101.572147,59,7.0,2098,-1
195860,2019-01-01 21:57:55,003000300000359769033400604,14.844158,101.590907,60,7.0,2098,-1


## get rid of mini cluster

In [ ]:
import collections
import pickle

pkl_filename = train_path+"/QBcluster_q1_treshold12.pkl"
with open(pkl_filename, 'rb') as file:
    clusters = pickle.load(file)

print("Nb. clusters:", len(clusters))
print("Cluster sizes:", map(len, clusters))
print("Small clusters:", collections.Counter(clusters < 2)[True], end=' /')
print("Small clusters:", clusters < 2)
print("Streamlines indices of the first cluster:\n", clusters[0].indices)
# print("Centroid of the last cluster:\n", clusters[-1].centroid)

is_mini_cluster = (clusters == 1)

mini_cluster = []
for ci in range(0, len(is_mini_cluster)):
  if is_mini_cluster[ci]:
    mini_cluster.append(ci)
print(len(mini_cluster)) 

df.loc[df.route_cluster.isin(mini_cluster), 'route_cluster'] = -1
df.loc[df.route_cluster == -1].sp.value_counts()

## X and Label shuffle

In [ ]:
state_names = []
uniq_r_cluster = df.route_cluster.unique()
for cluster in uniq_r_cluster:
  state_names.append(str(cluster))
state_names[:5]

X = []
labels = []
for month in months:
  for day in days:
    day_df = df[(df.day == day) & (df.month == month)]
    uniq_trips = day_df.trip_id.unique()
    for trip_id in uniq_trips:
      trip = day_df[day_df.trip_id==trip_id]
      X.append(trip.sp.to_numpy().astype(str))
      # labels.append(['None-start']+[str(l) for l in trip.route_cluster.to_list()])
      labels.append([str(l) for l in trip.route_cluster.to_list()])
print(X[0][:5])
print(labels[0][:5])
print(len(X))
print(len(labels))

In [ ]:
import random
random.seed(42)

X2 = X.copy()
labels2 = labels.copy()

temp = list(zip(X2, labels2))
random.shuffle(temp)
X2, labels2 = zip(*temp)
X2 = list(X2)
labels2 = list(labels2)

print(X[0][:5])
print(X2[0][:5])

print(labels[0][:5])
print(labels2[0][:5])

## train and test 10-fold cross validata

In [ ]:
from collections import Counter
import pickle

# pkl_filename = data_path+"/osm_json/dbscan.pkl"
# with open(pkl_filename, 'rb') as file:
#     db = pickle.load(file)
# sp_list = db.labels_
# sp_list = set(map(str, sp_list)) 

def score(model, seq, label):
  p = []
  prob,path = model.viterbi(seq)
  for s in path[1:]:
    p.append(s[1].name)
  
  # c = Counter(p)
  # tp = 0
  # if c[label]:
  #   tp = c[label]
  # total = sum(c.values())
  
  tp = 0
  for _state_name, _label in zip(p, label):
    if _state_name == _label:
      tp+=1
  total = len(label)


  # print("best prob : {}% ".format(np.exp(prob)*100))
  # print("beat path : {}".format(p))   
  # print("labels : {}".format(label))
  # print("sequen    : {}".format(seq))

  return tp/total

In [ ]:
from pomegranate import *

total = len(X2)
fold = int(total/10)
start = 0
starts = [] # [0, 952, 1904, 2856, 3808, 4760, 5712, 6664, 7616, 8568]
for i in range(10):
  starts.append(start)
  start+=fold

accuracy_train = []
accuracy_test = []
 
for start in starts: # 10
  X_train = X2[:start]+X2[start+fold:]
  X_test = X2[start:start+fold]
  labels_train = labels2[:start]+labels2[start+fold:]
  labels_test = labels2[start:start+fold]

  # บรรทัดนี้ปลอดภัยไว้ก่อน
  uniq_sp = set()
  for seq in X_train:
    for sp in seq:
      uniq_sp.add(sp)
  uniq_label = set()
  # for label in labels_train:
  for label in labels_train:
    for uniq in list(set(label)):
      uniq_label.add(uniq)

  new_label = []
  for label2 in labels_train:
    new_label.append(['None-start']+label2)
  labels_train = new_label
  
  unseen_sp = sp_list - uniq_sp
  # print(unseen_sp)
  X_train.append(np.array(list(unseen_sp)))
  labels_train.append(['None-start'] + ['-1'] * len(unseen_sp))

  model = HiddenMarkovModel.from_samples(
      DiscreteDistribution, 
      n_components=len(uniq_label), 
      X=X_train, 
      labels=labels_train,
      algorithm='labeled',
      state_names=list(uniq_label), 
      inertia=0.001,
      max_iterations=10,
      n_jobs=-1
    ) 

  accuracy_train_percent = []
  for percent in [0.25, 0.5, 0.75, 0.9]:
    accu = 0
    for seq, label in zip(X_train, labels_train):
      sc = score(model, np.array(seq[:int(len(seq)*percent)]), label[1:])
      if sc > 0:
        # accu += sc
        accu += 1
    train_score = accu/len(X_train)
    accuracy_train_percent.append(train_score)
  accuracy_train.append(accuracy_train_percent)

  accuracy_test_percent = []
  for percent in [0.25, 0.5, 0.75, 0.9]:
    accu = 0
    for seq, label in zip(X_test, labels_test):
      sc = score(model, np.array(seq[:int(len(seq)*percent)]), label[1:])
      if sc > 0:
        # accu += sc
        accu += 1
    test_score = accu/len(X_test)
    accuracy_test_percent.append(test_score)
  accuracy_test.append(accuracy_test_percent)

  print(f'Score {start} - {start+fold}')
  print(f'Train score')
  print(f'\t25% trip traverse : {accuracy_train_percent[0]}')
  print(f'\t50% trip traverse : {accuracy_train_percent[1]}')
  print(f'\t75% trip traverse : {accuracy_train_percent[2]}')
  print(f'\t90% trip traverse : {accuracy_train_percent[3]}')
  print(f'Test score')
  print(f'\t25% trip traverse : {accuracy_test_percent[0]}')
  print(f'\t50% trip traverse : {accuracy_test_percent[1]}')
  print(f'\t75% trip traverse : {accuracy_test_percent[2]}')
  print(f'\t90% trip traverse : {accuracy_test_percent[3]}')

print('all score')
print('train score: ')
for i in range(4):
  sum = 0
  for fold in accuracy_train:
    sum += fold[i]
  print(f'\ttrip traverse {i} : {sum/10}')

print('test score: ')
for i in range(4):
  sum = 0
  for fold in accuracy_test:
    sum += fold[i]
  print(f'\ttrip traverse {i} : {sum/10}')